<a href="https://colab.research.google.com/github/xiongsmart/triton/blob/main/triton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install triton

import torch

import triton
import triton.language as tl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 5.5 MB/s eta 0:00:00


In [ ]:
@triton.jit
def copy(x_ptr, y_ptr, block_size: tl.constexpr, N):
  block_id = tl.program_id(0)  # blockId.x
  offsets = block_id * block_size + tl.arange(0, block_size)
  mask = offsets < N
  x = tl.load(x_ptr + offsets, mask)
  tl.store(y_ptr + offsets, x, mask)

N = 1024
block_size = 128
x = torch.randn((N,), device = 'cuda')
y = torch.zeros((N,), device = 'cuda')
grid = triton.cdiv(N, block_size)
copy[(grid, )](x, y, block_size, N)

x

y

tensor([-0.5186,  0.0677, -0.3323,  ..., -0.2055,  0.3136, -2.4771],
       device='cuda:0')